In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import string
import re
from num2words import num2words

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True, nb_workers=16)

from functools import lru_cache

import pymorphy3
import random

morph = pymorphy3.MorphAnalyzer()
punctuation = string.punctuation

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
markup = {'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}

In [4]:
df_train = pd.read_parquet('train_ru-00000-of-00001.parquet')
df_test = pd.read_parquet('test_ru-00000-of-00001.parquet')
df_val = pd.read_parquet('val_ru-00000-of-00001.parquet')

In [5]:
df_main = pd.concat([df_train, df_test, df_val], ignore_index=True)

In [6]:
df_main.head()

,tokens,ner_tags,lang
0,"[Детство, провёл, в, Надьсомбате, ,, с, 1860, ...","[0, 0, 0, 5, 0, 0, 0, 0, 0]",ru
1,"[На, самом, деле, 6, августа, был, датирован, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]",ru
2,"[фунтов, .]","[7, 0]",ru
3,"[Исидор, Севильский, (, ок, .]","[1, 2, 0, 0, 0]",ru
4,"[Самыми, известными, являются, курганы, Накаод...","[0, 0, 0, 0, 5, 0, 0, 5, 0, 0, 5, 0, 5, 0, 0, ...",ru


In [7]:
# Function to check if number of tokens equals number of ner tags for a given row
def length_check(tokens, ner_tags):
    return 'ok' if len(tokens) == len(ner_tags) else 'WRONG'

In [8]:
df_main['tokens'] = df_main['tokens'].apply(lambda x: x.tolist())
df_main['ner_tags'] = df_main['ner_tags'].apply(lambda x: x.tolist())
df_main['len_check'] = df_main.apply(lambda row: length_check(row['tokens'], row['ner_tags']), axis=1)
df_main['augmented_tokens'] = df_main['tokens']
df_main['augmented_ner_tags'] = df_main['ner_tags']
df_main.drop('lang', inplace=True, axis=1)

In [9]:
df_main

,tokens,ner_tags,len_check,augmented_tokens,augmented_ner_tags
0,"[Детство, провёл, в, Надьсомбате, ,, с, 1860, ...","[0, 0, 0, 5, 0, 0, 0, 0, 0]",ok,"[Детство, провёл, в, Надьсомбате, ,, с, 1860, ...","[0, 0, 0, 5, 0, 0, 0, 0, 0]"
1,"[На, самом, деле, 6, августа, был, датирован, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]",ok,"[На, самом, деле, 6, августа, был, датирован, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]"
2,"[фунтов, .]","[7, 0]",ok,"[фунтов, .]","[7, 0]"
3,"[Исидор, Севильский, (, ок, .]","[1, 2, 0, 0, 0]",ok,"[Исидор, Севильский, (, ок, .]","[1, 2, 0, 0, 0]"
4,"[Самыми, известными, являются, курганы, Накаод...","[0, 0, 0, 0, 5, 0, 0, 5, 0, 0, 5, 0, 5, 0, 0, ...",ok,"[Самыми, известными, являются, курганы, Накаод...","[0, 0, 0, 0, 5, 0, 0, 5, 0, 0, 5, 0, 5, 0, 0, ..."
...,...,...,...,...,...
115435,"[За, день, до, боя, оба, боксёра, посетили, ве...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]",ok,"[За, день, до, боя, оба, боксёра, посетили, ве...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]"
115436,"[Согласно, боксёрским, традициям, ,, Али, как,...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]",ok,"[Согласно, боксёрским, традициям, ,, Али, как,...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
115437,"[Во, время, того, ,, как, рефери, напоминал, б...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",ok,"[Во, время, того, ,, как, рефери, напоминал, б...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."
115438,"[Виктория, является, вторым, по, площади, остр...","[5, 0, 0, 0, 0, 0, 0, 5, 0]",ok,"[Виктория, является, вторым, по, площади, остр...","[5, 0, 0, 0, 0, 0, 0, 5, 0]"


In [10]:
# All good at the start
df_main['len_check'].value_counts()

ok    115440
Name: len_check, dtype: int64

#### Now lets start augmenting the data to simulate transcribed speech:
- Changing random tokens to their lemma, to simulate transcribing a long, complicated word without the ending
- Removing non-alphanumeric chars, removing shortened tokens for thousands and millions, lowercasing
- If a token is a square or cube sign: $^{2}$ or $^{3}$, replace it with words
- Removing punctuation tokens

#### And of course, amend the ner_tokens accordingly


In [25]:
# Takes a list of tokens, and changes 1 or 2 (based on number of tokens)
# random tokens to its lemma, simulating transcribed speech system mistake,
# with the following conditions:
# -  original token should be at elast 7 characters long,
#    since transcribed speech system will probably recognize short words more accurately
# -  token should not start with uppercase, to aviod changing unique names
# -  the original token and its lemma must not differ in more than 5 characters.
#    since transcribed words should not be too different (assuming we arent using utter trash
#    transcription system.)

def simulate_mistakes(tokens):
    suitable_for_augmentation = []
    
    @lru_cache(maxsize=512)
    def get_lemma(word):
        parsed_word = morph.parse(word)[0]
        lemma = parsed_word.normal_form
        return lemma
    
    def lemmatize_tokens(tokens, suitable_for_augmentation, k):
        to_augment = random.sample(suitable_for_augmentation, k)
        for i in to_augment:
            tokens[i] = get_lemma(tokens[i])
    
    for i in range(len(tokens)):
        word = tokens[i]
        if len(word) > 6 and not word[0].isupper():
            lemma = get_lemma(word)
            if lemma != word:
                difference_counter = abs(len(word) - len(lemma))
                for index in range(min(len(word), len(lemma))):
                    if word[index] != lemma[index]:
                        difference_counter += 1

                if difference_counter < 6:
                        suitable_for_augmentation.append(i)
                    
    if suitable_for_augmentation:
        if len(tokens) <=10:
            lemmatize_tokens(tokens, suitable_for_augmentation, 1)
        elif len(suitable_for_augmentation) > 3:
            lemmatize_tokens(tokens, suitable_for_augmentation, 2)
       
    return tokens

In [12]:
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True, nb_workers=16)

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [13]:
df_main['augmented_tokens'] = df_main['augmented_tokens'].parallel_apply(lambda x: simulate_mistakes(x))

In [26]:
# Function that splits tokens by hyphen (since hyphen is not going to be in trascribed speech)
# and adjusts the ner_tags accordingly (iserting same tags, as the original token had)
def split_hyphen(tokens, ner_tags):
    
    for i in range(len(tokens)):
        if '-' in tokens[i] or '—' in tokens[i]:
            new_tokens = tokens[i].split('-')
            new_tokens = tokens[i].split('—')
            tag_to_insert = ner_tags[i]
            del ner_tags[i]
            del tokens[i] 
            index = i
            
            for _ in range(len(new_tokens)):
                ner_tags.insert(index, tag_to_insert)
                tokens.insert(index, new_tokens[_])
                index += 1 
               
    return tokens, ner_tags

In [27]:
df_main[['augmented_tokens', 'augmented_ner_tags']] = df_main.parallel_apply(lambda row: pd.Series(split_hyphen(row['tokens'], row['ner_tags'])), axis=1)

In [28]:
#quick check if ner tags are changed accordingly
df_main['augmented_length_check'] = df_main.apply(lambda row: length_check(row['augmented_tokens'], row['augmented_ner_tags']), axis=1)
df_main['augmented_length_check'].value_counts()

ok    115410
Name: augmented_length_check, dtype: int64

In [29]:
# Function to replace all non-alphanumric tokens with empty strings
# as well as change short tokens for kilometer, meter, thousand and million with full ones
# and also replace squaring and cubing signs with words
def cubes_squares_punct_etc(tokens, ner_tags):
    
    def switch_ner_tags(ner_tags, i):
        ner_tags[i-1], ner_tags[i] = ner_tags[i], ner_tags[i-1]
        
    def switch_tokens(tokens, i):
        tokens[i-1], tokens[i] = tokens[i], tokens[i-1]
        
    def m_km(tokens, i):
        if tokens[i-1] == 'километр':
            switch_tokens(tokens, i)
            switch_ner_tags(ner_tags, i)
        elif tokens[i-1] == 'метр':
            switch_tokens(tokens, i)
            switch_ner_tags(ner_tags, i)
    
    square, cube, half, third = '²', '³', '½', '⅓'
    thousand, million, billion = 'тыс', 'млн', 'млрд'
    km, m = 'км', 'м'
   
    for i in range(len(tokens)):
        tokens[i] = re.sub(r'[^\w\s]+', '', tokens[i])
        if tokens[i] == half: tokens[i] = 'с половиной'
        elif tokens[i] == third: tokens[i] = 'одна треть'
        elif tokens[i] == thousand: tokens[i] = 'тысяч'
        elif tokens[i] == million: tokens[i] = 'миллионов'
        elif tokens[i] == billion: tokens[i] = 'миллиадров'
        elif tokens[i] == km: tokens[i] = 'километр'
        elif tokens[i] == m: tokens[i] = 'метр'
        
        elif tokens[i] == square:
            tokens[i] = 'квадратный'
            m_km(tokens, i)
        
        elif tokens[i] == cube:
            tokens[i] = 'кубический'
            m_km(tokens, i)
            
    return tokens, ner_tags

In [30]:
df_main[['augmented_tokens', 'augmented_ner_tags']] = df_main.parallel_apply(lambda row: pd.Series(cubes_squares_punct_etc(row['augmented_tokens'], row['augmented_ner_tags'])), axis=1)

In [31]:
#check if we didnt break anything so far
df_main['augmented_length_check'] = df_main.apply(lambda row: length_check(row['augmented_tokens'], row['augmented_ner_tags']), axis=1)
df_main['augmented_length_check'].value_counts()

ok    115410
Name: augmented_length_check, dtype: int64

In [32]:
df_main

,tokens,ner_tags,len_check,augmented_tokens,augmented_ner_tags,augmented_length_check
0,"[Детство, провёл, в, Надьсомбате, ,, с, 1860, ...","[0, 0, 0, 5, 0, 0, 0, 0, 0]",ok,"[Детство, провёл, в, Надьсомбате, , с, 1860, г, ]","[0, 0, 0, 5, 0, 0, 0, 0, 0]",ok
1,"[На, самом, деле, 6, августа, был, датирован, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]",ok,"[На, самом, деле, 6, августа, был, датирован, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]",ok
2,"[фунтов, .]","[7, 0]",ok,"[фунтов, ]","[7, 0]",ok
3,"[Исидор, Севильский, (, ок, .]","[1, 2, 0, 0, 0]",ok,"[Исидор, Севильский, , ок, ]","[1, 2, 0, 0, 0]",ok
4,"[Самыми, известными, являются, курганы, Накаод...","[0, 0, 0, 0, 5, 0, 0, 5, 0, 0, 5, 0, 5, 0, 0, ...",ok,"[Самыми, известными, являются, курганы, Накаод...","[0, 0, 0, 0, 5, 0, 0, 5, 0, 0, 5, 0, 5, 0, 0, ...",ok
...,...,...,...,...,...,...
115405,"[За, день, до, боя, оба, боксёра, посетили, ве...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]",ok,"[За, день, до, боя, оба, боксёра, посетили, ве...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]",ok
115406,"[Согласно, боксёрским, традициям, ,, Али, как,...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]",ok,"[Согласно, боксёрским, традициям, , Али, как, ...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]",ok
115407,"[Во, время, того, ,, как, рефери, напоминал, б...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",ok,"[Во, время, того, , как, рефери, напоминал, бо...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",ok
115408,"[Виктория, является, вторым, по, площади, остр...","[5, 0, 0, 0, 0, 0, 0, 5, 0]",ok,"[Виктория, является, вторым, по, площади, остр...","[5, 0, 0, 0, 0, 0, 0, 5, 0]",ok


In [33]:
def filter_non_words(tokens, ner_tags):
    
    # Remove 'г's, using that just in case there are multiple occurences in one row
    if 'г' in tokens:
        g_indexes = [index for index, value in enumerate(tokens) if value == 'г']
        tokens = [value for index, value in enumerate(tokens) if index not in g_indexes]
        
    filtered_tags = [tag if token.isalnum() else np.nan for token, tag in zip(tokens, ner_tags)]
    filtered_tokens = [token for token in tokens if token != '' ]
    return filtered_tokens, [tag for tag in filtered_tags if pd.notnull(tag)]

In [34]:
df_main[['augmented_tokens', 'augmented_ner_tags']] = df_main.parallel_apply(lambda row: pd.Series(filter_non_words(row['augmented_tokens'], row['augmented_ner_tags'])), axis=1)

In [35]:
df_main['augmented_length_check'] = df_main.apply(lambda row: length_check(row['augmented_tokens'], row['augmented_ner_tags']), axis=1)
df_main['augmented_length_check'].value_counts()

ok    115410
Name: augmented_length_check, dtype: int64

In [36]:
df_main.drop(df_main[df_main['augmented_length_check'] == 'WRONG'].index, inplace=True)
df_main.reset_index(drop=True, inplace=True)

In [37]:
df_main

,tokens,ner_tags,len_check,augmented_tokens,augmented_ner_tags,augmented_length_check
0,"[Детство, провёл, в, Надьсомбате, ,, с, 1860, ...","[0, 0, 0, 5, 0, 0, 0, 0, 0]",ok,"[Детство, провёл, в, Надьсомбате, с, 1860]","[0, 0, 0, 5, 0, 0]",ok
1,"[На, самом, деле, 6, августа, был, датирован, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]",ok,"[На, самом, деле, 6, августа, был, датирован, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",ok
2,"[фунтов, .]","[7, 0]",ok,[фунтов],[7],ok
3,"[Исидор, Севильский, (, ок, .]","[1, 2, 0, 0, 0]",ok,"[Исидор, Севильский, ок]","[1, 2, 0]",ok
4,"[Самыми, известными, являются, курганы, Накаод...","[0, 0, 0, 0, 5, 0, 0, 5, 0, 0, 5, 0, 5, 0, 0, ...",ok,"[Самыми, известными, являются, курганы, Накаод...","[0, 0, 0, 0, 5, 0, 0, 5, 0, 5, 0, 5, 0, 0, 5]",ok
...,...,...,...,...,...,...
115405,"[За, день, до, боя, оба, боксёра, посетили, ве...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]",ok,"[За, день, до, боя, оба, боксёра, посетили, ве...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",ok
115406,"[Согласно, боксёрским, традициям, ,, Али, как,...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]",ok,"[Согласно, боксёрским, традициям, Али, как, пр...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]",ok
115407,"[Во, время, того, ,, как, рефери, напоминал, б...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",ok,"[Во, время, того, как, рефери, напоминал, бокс...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, ...",ok
115408,"[Виктория, является, вторым, по, площади, остр...","[5, 0, 0, 0, 0, 0, 0, 5, 0]",ok,"[Виктория, является, вторым, по, площади, остр...","[5, 0, 0, 0, 0, 0, 0, 5]",ok


In [38]:
def replace_numerical_with_words(tokens, ner_tags):
    adjusted_ner_tags = []
    
    for i, token in enumerate(tokens):
        if token.isnumeric():
            index = i
            new_tokens = num2words(int(token), lang='ru').split()
            tag_to_insert = ner_tags[i]
            count = len(new_tokens)
            del ner_tags[i]
            del tokens[i] 
            for _ in range(len(new_tokens)):
                ner_tags.insert(index, tag_to_insert)
                tokens.insert(index, new_tokens[_])
                index += 1 
                
    return tokens, ner_tags

In [39]:
def look_for_trash(trash):
    for i in range(len(df_main)):
        if trash in df_main['augmented_tokens'][i]:
            print(i)
            print(df_main['augmented_tokens'][i])

In [40]:
trash_2 = '₂'
look_for_trash(trash_2)

91574
['Например', 'для', 'получения', 'удовлетворительного', 'результата', 'необходимо', 'добавить', 'вещества', 'вызывающие', 'усиление', 'реакции', 'или', 'прибавить', 'флюсы', 'такие', 'как', 'плавиковый', 'шпат', 'CaF', '₂', 'или', 'сплавить', 'восстановляемые', 'окислы', 'с', 'энергично', 'действующими', 'металлами', 'или', 'как', 'при', 'восстановлении', 'хрома', 'прибавлять', 'хромовокислые', 'соли']


In [41]:
df_main['augmented_tokens'][91574].index(trash_2)

19

In [42]:
del df_main['augmented_tokens'][91574][19]
del df_main['augmented_ner_tags'][91574][19]

In [43]:
trash_1 = '₃'
look_for_trash(trash_1)

69622
['Для', 'олеума', 'минимальное', 'ρ', 'при', 'концентрации', '10', 'SO', '₃']


In [44]:
df_main['augmented_tokens'][69622]

['Для', 'олеума', 'минимальное', 'ρ', 'при', 'концентрации', '10', 'SO', '₃']

In [45]:
del df_main['augmented_tokens'][69622][8]
del df_main['augmented_ner_tags'][69622][8]

In [46]:
trash_3 = '¼'
look_for_trash(trash_3)

114169
['Кроме', 'того', 'в', 'составе', 'батальона', 'присутствовало', 'медицинское', 'подразделение', 'в', 'составе', '11', 'человек', 'имевшее', 'бронетранспортёр', 'M3', 'в', 'санитарном', 'варианте', 'и', '¼', 'тонный', 'автомобиль', 'повышенной', 'проходимости', 'с', '1тонным', 'прицепом']


In [47]:
df_main['augmented_tokens'][114169][19] = 'четверть'

In [48]:
trash_4 = '¹'
look_for_trash(trash_4)

13808
['Наси', 'самоназвание', '¹', 'na', 'квадратный', 'khi', 'народ', 'в', 'Китае', 'населяющий', 'предгорья', 'Гималаев']


In [49]:
del df_main['augmented_tokens'][13808][2]
del df_main['augmented_ner_tags'][13808][2]

In [50]:
df_main[['augmented_tokens', 'augmented_ner_tags']] = df_main.parallel_apply(lambda row: pd.Series(replace_numerical_with_words(row['augmented_tokens'], row['augmented_ner_tags'])), axis=1)

In [51]:
df_main

,tokens,ner_tags,len_check,augmented_tokens,augmented_ner_tags,augmented_length_check
0,"[Детство, провёл, в, Надьсомбате, ,, с, 1860, ...","[0, 0, 0, 5, 0, 0, 0, 0, 0]",ok,"[Детство, провёл, в, Надьсомбате, с, одна, тыс...","[0, 0, 0, 5, 0, 0, 0, 0, 0]",ok
1,"[На, самом, деле, 6, августа, был, датирован, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]",ok,"[На, самом, деле, шесть, августа, был, датиров...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",ok
2,"[фунтов, .]","[7, 0]",ok,[фунтов],[7],ok
3,"[Исидор, Севильский, (, ок, .]","[1, 2, 0, 0, 0]",ok,"[Исидор, Севильский, ок]","[1, 2, 0]",ok
4,"[Самыми, известными, являются, курганы, Накаод...","[0, 0, 0, 0, 5, 0, 0, 5, 0, 0, 5, 0, 5, 0, 0, ...",ok,"[Самыми, известными, являются, курганы, Накаод...","[0, 0, 0, 0, 5, 0, 0, 5, 0, 5, 0, 5, 0, 0, 5]",ok
...,...,...,...,...,...,...
115405,"[За, день, до, боя, оба, боксёра, посетили, ве...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]",ok,"[За, день, до, боя, оба, боксёра, посетили, ве...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",ok
115406,"[Согласно, боксёрским, традициям, ,, Али, как,...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]",ok,"[Согласно, боксёрским, традициям, Али, как, пр...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]",ok
115407,"[Во, время, того, ,, как, рефери, напоминал, б...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",ok,"[Во, время, того, как, рефери, напоминал, бокс...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, ...",ok
115408,"[Виктория, является, вторым, по, площади, остр...","[5, 0, 0, 0, 0, 0, 0, 5, 0]",ok,"[Виктория, является, вторым, по, площади, остр...","[5, 0, 0, 0, 0, 0, 0, 5]",ok


In [52]:
df_main['augmented_length_check'] = df_main.apply(lambda row: length_check(row['augmented_tokens'], row['augmented_ner_tags']), axis=1)
df_main['augmented_length_check'].value_counts()

ok    115410
Name: augmented_length_check, dtype: int64

In [53]:
def lowercasing(tokens):
    return [token.lower() for token in tokens]

In [54]:
df_main['augmented_tokens'] = df_main['augmented_tokens'].parallel_apply(lambda x: lowercasing(x))

In [55]:
df_main.to_csv('Wikineural_augmented.csv', index=False)